### Парсим ссылки на квартиры

Мы взяли в качестве источника данных рынок арендного жилья в Нижнем Новгороде, так как это достаточно небольшой город, всего получилось ~1500 квартир (их число постоянно меняется), но в нем есть метро, что позволяет собрать больше данных. Парсили Циан с использованием Tor и Selenium.

In [1]:
import requests  
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm
import time
from fake_useragent import UserAgent
from joblib import Parallel, delayed
import numpy as np
import pandas as pd
import ast
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

In [25]:
import time
from fake_useragent import UserAgent
from joblib import Parallel, delayed
import numpy as np
import pandas as pd
import ast

In [13]:
from selenium.webdriver.chrome.options import Options
from fake_useragent import UserAgent

opts = Options()
opts.add_argument(f"user-agent={UserAgent(verify_ssl=False).chrome}")
driver = webdriver.Chrome(executable_path=r"C:\Users\ilja_\.wdm\drivers\chromedriver\win32\91.0.4472.19\chromedriver.exe", options=opts)

In [3]:
from webdriver_manager.chrome import ChromeDriverManager

In [4]:
driver = webdriver.Chrome(ChromeDriverManager().install())



====== WebDriver manager ======
Current google-chrome version is 91.0.4472
Get LATEST driver version for 91.0.4472
Driver [C:\Users\ilja_\.wdm\drivers\chromedriver\win32\91.0.4472.19\chromedriver.exe] found in cache


In [30]:
def checkIP():
    ip = requests.get('http://checkip.dyndns.org').content
    soup = BeautifulSoup(ip, 'html.parser')
    print(soup.find('body').text)

checkIP()

Current IP Address: 185.220.102.253


In [45]:
import socks
import socket


socks.set_default_proxy(socks.SOCKS5, "localhost", 9150)
socket.socket = socks.socksocket

checkIP()

Current IP Address: 109.70.100.49


In [63]:
path_head = 'https://nn.cian.ru/cat.php?deal_type=rent&engine_version=2&offer_type=flat&p='
path_tail = '&region=4885&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1&type=4'

In [55]:
links = []
for page in tqdm(range(1,53)):
    path = path_head + str(page) + path_tail
    driver.get(path)
    #time.sleep(0.5)
    #response = requests.get(path, headers={'User-Agent': UserAgent().chrome})
    tree = BeautifulSoup(driver.page_source, 'html.parser')
    linkareas = tree.find_all('div', {'data-name' : 'LinkArea'})
    for linkarea in linkareas:
        links.append(linkarea.find('a').get('href'))

In [64]:
print(len(links))

1457


### Парсим информацию о квартирах

In [45]:
def new_flat(tree):
    """Принимает в качестве параметра html-дерево. С дерева собирает необходимую информацию. Возвращает словарь."""
    flat = {}
    flat['roomsCount'] = tree.find_all('div', {'data-name' : 'OfferTitle'})[0].find('h1').text.split('.')[0]
    flat['price'] = tree.find_all('span', {'itemprop' : 'price'})[0].get('content')
    flat['address'] = tree.find_all('div', {'data-name' : 'Geo'})[0].find_all('span')[0].get('content')
    try:
        flat['tel'] = tree.find_all('div', {'data-name' : 'OfferContactsAside'})[0].find('a').get('href')
    except:
        pass
    try:
        flat['underground'] = tree.find_all('li', {'data-name' : 'renderUnderground'})[0].find('a').text
        flat['undergroundTime'] = tree.find_all('li', {'data-name' : 'renderUnderground'})[0].find('span').text[4:]
    except:
        pass
    try:
        flat['deposit'], flat['comission'], flat['prepay'], _ =  tree.find_all('div', 
                    {'data-name' : 'OfferTerms'})[0].find('p').text.replace('\xa0', ' ').split(', ')
    except:
        pass
    try:    
        flat['total_area'] = tree.find_all('div', text='Общая')[0].find_previous('div').text.replace('\xa0', ' ')
    except:
        pass
    try:
        flat['living_area'] = tree.find_all('div', text='Жилая')[0].find_previous('div').text.replace('\xa0', ' ')
    except:
        pass
    try:
        flat['kitchen_area'] = tree.find_all('div', text='Кухня')[0].find_previous('div').text.replace('\xa0', ' ')
    except:
        pass
    try:
        flat['floor'] = tree.find_all('div', text='Этаж')[0].find_previous('div').text.replace('\xa0', ' ')
    except:
        pass
    try:
        flat['build_year'] = tree.find_all('div', text='Построен')[0].find_previous('div').text.replace('\xa0', ' ')
    except:
        pass
    try:
        tmp = str(tree.find_all('script', {'type' : 'text/javascript'}))
        i1 = tmp.find('coordinates')
        i2 = tmp.find('}', i1)
        tmp_dict = ast.literal_eval(tmp[i1:i2+1].replace('coordinates":', ''))
        flat['lng'] = tmp_dict['lng']
        flat['lat'] = tmp_dict['lat']
    except:
        pass
    try:
        flat['description'] = tree.find_all('div', {'data-name' : 'Description'})[0].find('p').text
    except:
        pass
    return flat

In [54]:
def parser(n):
    """Принимает в качестве параметра номер ссылки из списка links.
    По ссылке выгружает html-дерево. Возвращает словарь """
    try:
        driver.get(links[n])
        html = driver.page_source
    except:
        return None
    time.sleep(0.5)
    tree = BeautifulSoup(html, 'html.parser')
    n_jobs=2
    flat = new_flat(tree)
    flat['link'] = links[n]
    #time.sleep(1)
    return flat

In [56]:
list = [parser(n) for n in tqdm(range(len(links)))]

In [65]:
df = pd.DataFrame(list)
df.head(2)

,roomsCount,price,address,tel,underground,undergroundTime,deposit,comission,prepay,total_area,living_area,kitchen_area,floor,build_year,lng,lat,description,link
0,1-комн,30 000 ₽/мес.,"Нижегородская область, Нижний Новгород, р-н Ни...",tel:+79587282362,Горьковская,24 мин. пешком,Залог 20 000 ₽,комиссия 60%,предоплата за 1 месяц,40 м²,20 м²,11 м²,5 из 9,1976,44.014134,56.326148,Сдается очень уютная и современная квартира ст...,https://nn.cian.ru/rent/flat/258002335/
1,1-комн,18 000 ₽/мес.,"Нижегородская область, Нижний Новгород, р-н Ка...",tel:+79587281852,Стрелка,23 мин. пешком,Без залога,комиссия 60%,предоплата за 1 месяц,38 м²,20 м²,9 м²,6 из 12,2005,43.930699,56.346951,Сдаётся 1 комнатная квартира - студия. Только ...,https://nn.cian.ru/rent/flat/258046577/


In [66]:
df.to_csv('raw_nino_flats_dataset.csv', index=False)

In [67]:
dff = pd.read_csv('raw_nino_flats_dataset.csv')

In [68]:
dff.head(2)

,roomsCount,price,address,tel,underground,undergroundTime,deposit,comission,prepay,total_area,living_area,kitchen_area,floor,build_year,lng,lat,description,link
0,1-комн,30 000 ₽/мес.,"Нижегородская область, Нижний Новгород, р-н Ни...",tel:+79587282362,Горьковская,24 мин. пешком,Залог 20 000 ₽,комиссия 60%,предоплата за 1 месяц,40 м²,20 м²,11 м²,5 из 9,1976.0,44.014134,56.326148,Сдается очень уютная и современная квартира ст...,https://nn.cian.ru/rent/flat/258002335/
1,1-комн,18 000 ₽/мес.,"Нижегородская область, Нижний Новгород, р-н Ка...",tel:+79587281852,Стрелка,23 мин. пешком,Без залога,комиссия 60%,предоплата за 1 месяц,38 м²,20 м²,9 м²,6 из 12,2005.0,43.930699,56.346951,Сдаётся 1 комнатная квартира - студия. Только ...,https://nn.cian.ru/rent/flat/258046577/
